# Step 01: Initialize Environment

This notebook validates the environment setup and prepares the workspace for the analysis cycle.

**Tasks:**
- Validate database connectivity
- Display cycle configuration
- Verify working directories
- Initialize step execution tracking

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.cycle import get_cycle_by_name
from helpers.database import execute_query
from helpers.step import Step

## 1) Validate Cycle Status

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_01_Initialize_Environment.ipynb')

# Display cycle configuration
ux.header("Cycle Configuration")
ux.subheader("Work Context")
context_info = [
    ["Cycle Name", context.cycle_name],
    ["Stage", context.stage_name],
    ["Step", context.step_name],
    ["Notebook Path", str(context.notebook_path)]
]
ux.table(context_info, headers=["Property", "Value"])

# Retrieve and display cycle details from database
try:
    cycle = get_cycle_by_name(context.cycle_name)
    
    ux.subheader("Cycle Details")
    cycle_info = [
        ["ID", cycle['id']],
        ["Name", cycle['cycle_name']],
        ["Status", cycle['status']],
        ["Created", cycle['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
    ]
    ux.table(cycle_info, headers=["Property", "Value"])
    
    if cycle['status'] != 'ACTIVE':
        ux.warning(f"⚠ Cycle status is '{cycle['status']}' (expected 'ACTIVE')")
    else:
        ux.success("✓ Cycle is active and ready")
        
except Exception as e:
    ux.error(f"✗ Failed to retrieve cycle information: {str(e)}")
    raise

ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

## 2) Initialize Step Tracking

In [ ]:
# Validate cycle status
ux.header("Cycle Status Validation")

if cycle['status'] != 'ACTIVE':
    ux.warning(f"⚠ Cycle status is '{cycle['status']}' (expected 'ACTIVE')")
    step.log(f"Warning: Cycle status is '{cycle['status']}' instead of 'ACTIVE'", level="WARNING")
else:
    ux.success(f"✓ Cycle '{context.cycle_name}' is active and ready")
    step.log(f"Cycle '{context.cycle_name}' validated with status: {cycle['status']}")

## 3) Validate Database Connectivity

In [ ]:
# Test database connection
ux.header("Database Connectivity Check")

try:
    result = execute_query("SELECT version() as version")
    if not result.empty:
        db_version = result.iloc[0]['version']
        ux.success("✓ Database connection successful")
        ux.info(f"PostgreSQL Version: {db_version.split(',')[0]}")
        
        # Log to step tracker
        step.log(f"Database connectivity validated: PostgreSQL {db_version.split(',')[0]}")
    else:
        ux.error("✗ Database query returned no results")
        raise Exception("Database query returned empty result")
except Exception as e:
    ux.error(f"✗ Database connection failed: {str(e)}")
    raise

## 4) Validate Working Directories

In [ ]:
# Display working directory structure
ux.header("Working Directories")

# Get key directories from context
cycle_dir = context.cycle_directory
files_dir = cycle_dir / "files"
excel_config_dir = files_dir / "configuration"
data_dir = files_dir / "data"
logs_dir = cycle_dir / "logs"

# Check and display directory status
directories = [
    ["Cycle Directory", cycle_dir, cycle_dir.exists()],
    ["Files Directory", files_dir, files_dir.exists()],
    ["Excel Configuration", excel_config_dir, excel_config_dir.exists()],
    ["Data Directory", data_dir, data_dir.exists()],
    ["Logs Directory", logs_dir, logs_dir.exists()]
]

dir_info = []
all_exist = True
for name, path, exists in directories:
    status = "✓" if exists else "✗"
    dir_info.append([name, str(path), status])
    if not exists:
        all_exist = False

ux.table(dir_info, headers=["Directory", "Path", "Exists"])

if all_exist:
    ux.success("✓ All working directories are present")
    step.log(f"Directory structure validated: All {len(directories)} directories exist")
else:
    ux.warning("⚠ Some directories are missing (they will be created as needed)")
    existing_count = len([d for d in directories if d[2]])
    step.log(f"Directory structure checked: {existing_count} of {len(directories)} directories exist", level="WARNING")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

# Complete the step with output data
step.complete({
    'database_connected': True,
    'database_version': db_version,
    'cycle_name': context.cycle_name,
    'cycle_id': cycle['id'],
    'cycle_status': cycle['status'],
    'directories_validated': all_exist,
    'directories_count': len(directories),
    'directories_existing': len([d for d in directories if d[2]])
})

ux.success("\n" + "="*60)
ux.success("✓ ENVIRONMENT INITIALIZATION COMPLETED SUCCESSFULLY")
ux.success("="*60)
ux.info("\nYou can now proceed to Step 02: Validate Configuration File")